In [17]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt  # primeiro importamos a biblioteca para visualização
import numpy as np  # importamos também a biblioteca NumPy que irá nos fornecer diversos métodos para trabalhar com arrays
import seaborn as sns

#funcao auxiliar de conversao
def converte_categorias(df):
    pd.options.mode.chained_assignment = None  # default='warn'
    # job
    df.job = pd.Categorical(df.job)
    df['job'] = df.job.cat.codes
    # marital
    df.marital = pd.Categorical(df.marital)
    df['marital'] = df.marital.cat.codes
    # education
    df.education = pd.Categorical(df.education)
    df['education'] = df.education.cat.codes
    # default
    df.default = pd.Categorical(df.default)
    df['default'] = df.default.cat.codes
    # housing
    df.housing = pd.Categorical(df.housing)
    df['housing'] = df.housing.cat.codes
    # loan
    df.loan = pd.Categorical(df.loan)
    df['loan'] = df.loan.cat.codes
    # contact
    df.contact = pd.Categorical(df.contact)
    df['contact'] = df.contact.cat.codes
    # month
    df.month = pd.Categorical(df.month)
    df['month'] = df.month.cat.codes
    # outcome
    df.poutcome = pd.Categorical(df.poutcome)
    df['poutcome'] = df.poutcome.cat.codes
    return df

#dataset
trainset = pd.read_csv("data/trainset.csv")
testset = pd.read_csv("data/testset.csv")

X_train = trainset.loc[:, trainset.columns != "y"]
y_train = trainset.loc[:, trainset.columns == "y"]
y_train = y_train.values.ravel()


X_test = testset.loc[:, testset.columns != "y"]
y_test = testset.loc[:, testset.columns == "y"]
y_test = y_test.values.ravel()
#trainset

#visualização está ok
#sns.pairplot(trainset, hue="y")  # o parametro 'hue' diz qual coluna contém o alvo para distribuir as cores

#convertendo os dados para numérico
X_train = converte_categorias(X_train)
X_test = converte_categorias(X_test)
#X_train



#importando os arquivos necessários
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report


# from sklearn.tree import DecisionTreeClassifier

from sklearn.neighbors import KNeighborsClassifier
# from sklearn.naive_bayes import GaussianNB
# from sklearn.linear_model import LogisticRegression
# from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

folds = 3  # precisa ser maior que 2 - Interessante entre 5 e 10

# aqui nós criamos um objeto dicionário com o nome do hyperparâmetro e os possíveis valores que ele pode assumir
# os hyperparâmetros são específicos para cada algoritmo! Note como neste caso nós chamamos a classe do algoritmo
# sem passarmos os hyperparâmetros default

#KNeighbors
knn = KNeighborsClassifier()
knn_params = {
    "n_neighbors": [2, 3, 5, 10],
    "weights": ["uniform", "distance"],
    "p": [1, 2]
}

#MLPerceptron
pcptron = MLPClassifier()
pcptron_params = {
    "hidden_layer_sizes": [1,2],    
    "max_iter": [50,100,200]    
}

#RandomForest
rf = RandomForestClassifier()
rf_params = {
    "n_estimators": [5,10],
    "criterion": ["entropy"],
    "max_features": ["auto","sqrt"]
}


# O scikit learn e uma biblioteca bastante flexível. Com poucas linhas de código, podemos executar o mesmo experimento
# para diversos algoritmos: basta criar uma lista contendo os algoritmos e uma segunda lista contendo os 
# dicionários de hyperparâmetros. Detalhe: a ordem das listas é importante!
classifiers = [knn,rf]
grids = [knn_params,rf_params]

# gera uma lista de tuplas entre classifiers e grids para que cada um fique na
# posição correta [(class.1, parms.1), (class.2, params.2), ...]
grid_params = zip(classifiers, grids)

# aqui fazemos a busca - neste caso a busca é por força bruta, ou seja, vai
# testar todas as combinações que incluirmos no dicionário de parâmetros - há
# também a opção de se buscar randomicamente, mas precisariamos definir
# distribuições ao invés de parâmetros e os resultados são parecidos.
# a busca vai ser feita pelo ``score'' que definirmos



for _, (classifier, params) in enumerate(grid_params):

    print("Buscando para algoritmo: {0}\n".format(classifier.__class__))
    
    
    clf = GridSearchCV(estimator=classifier,  # algoritmo em teste
                               param_grid=params,  # parâmetros de busca
                               cv=folds,  # objeto que vai gerar as divisões
                               n_jobs=-1, #processamento
                               scoring='accuracy')  # score que será utilizado
        
    
    
    clf.fit(X_train, y_train.ravel())

    # aqui nós imprimimos o resultado - o método report vai imprimir as ``top''
    # melhores combinações encontrada na busca. Os parâmetros impressos
    # são aqueles que teríamos que usar para gerar o classificador de forma isolada
    print("Melhor seleção de hyperparâmetros:\n")
    print(clf.best_params_)
    print("\nScores (% de acertos) nos folds de validação:\n")
    means = clf.cv_results_['mean_test_score'] #clf
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("{:.3f} (+/-{:.3f}) for {}".format(mean, std * 2, params))

    print("\nResultado detalhado para o melhor modelo:\n")
    y_true, y_pred = y_test, rfecv.predict(X_test)
    print(classification_report(y_true, y_pred))






Buscando para algoritmo: <class 'sklearn.neighbors.classification.KNeighborsClassifier'>

Melhor seleção de hyperparâmetros:

{'n_neighbors': 10, 'p': 2, 'weights': 'uniform'}

Scores (% de acertos) nos folds de validação:

0.810 (+/-0.152) for {'n_neighbors': 2, 'p': 1, 'weights': 'uniform'}
0.730 (+/-0.182) for {'n_neighbors': 2, 'p': 1, 'weights': 'distance'}
0.815 (+/-0.154) for {'n_neighbors': 2, 'p': 2, 'weights': 'uniform'}
0.737 (+/-0.179) for {'n_neighbors': 2, 'p': 2, 'weights': 'distance'}
0.781 (+/-0.174) for {'n_neighbors': 3, 'p': 1, 'weights': 'uniform'}
0.778 (+/-0.175) for {'n_neighbors': 3, 'p': 1, 'weights': 'distance'}
0.792 (+/-0.172) for {'n_neighbors': 3, 'p': 2, 'weights': 'uniform'}
0.789 (+/-0.172) for {'n_neighbors': 3, 'p': 2, 'weights': 'distance'}
0.804 (+/-0.165) for {'n_neighbors': 5, 'p': 1, 'weights': 'uniform'}
0.801 (+/-0.168) for {'n_neighbors': 5, 'p': 1, 'weights': 'distance'}
0.811 (+/-0.166) for {'n_neighbors': 5, 'p': 2, 'weights': 'uniform'}
0

NotFittedError: This RFECV instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.